In [1]:
import numpy as np
import pandas as pd

import os

import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn import cross_validation, metrics

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import defaultdict

//anaconda/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('./data/home-credit-default-risk/application_train.csv')

test = pd.read_csv("./data/home-credit-default-risk/application_test.csv")
bureau = pd.read_csv("./data/home-credit-default-risk/bureau.csv")
previous_app  = pd.read_csv("./data/home-credit-default-risk/previous_application.csv")
bureau_balance = pd.read_csv("./data/home-credit-default-risk/bureau_balance.csv")
credit_cb = pd.read_csv("./data/home-credit-default-risk/credit_card_balance.csv")
installments_pay = pd.read_csv("./data/home-credit-default-risk/installments_payments.csv")
POS_CASH_b = pd.read_csv("./data/home-credit-default-risk/POS_CASH_balance.csv")

ID = test.SK_ID_CURR

In [48]:
previous_app  = pd.read_csv("./data/home-credit-default-risk/previous_application.csv")

In [3]:
agg_funs = {'SK_ID_CURR': 'count', 'AMT_CREDIT': 'sum'}

In [4]:
previous_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
prev_app_agg = previous_app.groupby('SK_ID_CURR').agg(agg_funs)

In [7]:
prev_app_agg.head()

,SK_ID_CURR,AMT_CREDIT
SK_ID_CURR,,
100001,1,23787.0
100002,1,179055.0
100003,3,1452573.0
100004,1,20106.0
100005,2,40153.5


In [8]:
prev_app_agg.columns = ['PREV APP COUNT', 'TOTAL PREV LOAN AMT']

In [9]:
prev_app_agg.head()

,PREV APP COUNT,TOTAL PREV LOAN AMT
SK_ID_CURR,,
100001,1,23787.0
100002,1,179055.0
100003,3,1452573.0
100004,1,20106.0
100005,2,40153.5


In [14]:
merged_df = train.merge(prev_app_agg, left_on='SK_ID_CURR', right_index=True, how='left')

In [13]:
prev_app_agg[prev_app_agg.index == 100002]

,PREV APP COUNT,TOTAL PREV LOAN AMT
SK_ID_CURR,,
100002,1,179055.0


In [15]:
merged_df[merged_df['SK_ID_CURR'] == 100002].transpose()

,0
SK_ID_CURR,100002
TARGET,1
NAME_CONTRACT_TYPE,Cash loans
CODE_GENDER,M
FLAG_OWN_CAR,N
FLAG_OWN_REALTY,Y
CNT_CHILDREN,0
AMT_INCOME_TOTAL,202500
AMT_CREDIT,406598
AMT_ANNUITY,24700.5


In [17]:
previous_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
prev_app_mean = previous_app.groupby('SK_ID_CURR').mean()

In [21]:
merged_df = merged_df.merge(
    prev_app_mean, left_on='SK_ID_CURR', right_index=True,
    how='left', suffixes=['', '_PAVG'])

In [22]:
merged_df.head().transpose()

,0,1,2,3,4
SK_ID_CURR,100002,100003,100004,100006,100007
TARGET,1,0,0,0,0
NAME_CONTRACT_TYPE,Cash loans,Cash loans,Revolving loans,Cash loans,Cash loans
CODE_GENDER,M,F,M,F,M
FLAG_OWN_CAR,N,N,Y,N,N
FLAG_OWN_REALTY,Y,N,Y,Y,Y
CNT_CHILDREN,0,0,0,0,0
AMT_INCOME_TOTAL,202500,270000,67500,135000,121500
AMT_CREDIT,406598,1.2935e+06,135000,312682,513000
AMT_ANNUITY,24700.5,35698.5,6750,29686.5,21865.5


In [23]:
merged_df.shape

(307511, 144)

In [24]:
def process_dataframe(input_df, encoder_dict=None):
    """ Process a dataframe into a form useable by LightGBM """

    # Label encode categoricals
    print('Label encoding categorical features...')
    categorical_feats = input_df.columns[input_df.dtypes == 'object']
    categorical_feats = categorical_feats
    encoder_dict = {}
    for feat in categorical_feats:
        encoder = LabelEncoder()
        input_df[feat] = encoder.fit_transform(input_df[feat].fillna('NULL'))
        encoder_dict[feat] = encoder
    print('Label encoding complete.')

    return input_df, categorical_feats.tolist(), encoder_dict

In [25]:
prev_app_df, cat_feats, _ = process_dataframe(previous_app)

Label encoding categorical features...
Label encoding complete.


In [26]:
cat_feats

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION']

In [28]:
prev_app_df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,1,1730.430,17145.0,17145.0,0.0,17145.0,2,15,...,2,12.0,4,14,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,0,25188.615,607500.0,679671.0,NaN,607500.0,4,11,...,10,36.0,2,7,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,0,15060.735,112500.0,136444.5,NaN,112500.0,5,11,...,10,12.0,1,6,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,0,47041.335,450000.0,470790.0,NaN,450000.0,1,7,...,10,12.0,4,8,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,0,31924.395,337500.0,404055.0,NaN,337500.0,4,9,...,10,24.0,1,3,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
previous_app.head().transpose()

,0,1,2,3,4
SK_ID_PREV,2030495,2802425,2523466,2819243,1784265
SK_ID_CURR,271877,108129,122040,176158,202054
NAME_CONTRACT_TYPE,Consumer loans,Cash loans,Cash loans,Cash loans,Cash loans
AMT_ANNUITY,1730.43,25188.6,15060.7,47041.3,31924.4
AMT_APPLICATION,17145,607500,112500,450000,337500
AMT_CREDIT,17145,679671,136444,470790,404055
AMT_DOWN_PAYMENT,0,NaN,NaN,NaN,NaN
AMT_GOODS_PRICE,17145,607500,112500,450000,337500
WEEKDAY_APPR_PROCESS_START,SATURDAY,THURSDAY,TUESDAY,MONDAY,THURSDAY
HOUR_APPR_PROCESS_START,15,11,11,7,9


In [35]:
categorical_feats = previous_app.columns[previous_app.dtypes == 'object']

In [36]:
categorical_feats

Index(['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')

In [39]:
for feat in categorical_feats:
    encoder = LabelEncoder()
    previous_app[feat] = encoder.fit_transform(previous_app[feat].fillna('NULL'))

In [40]:
previous_app.tail().transpose()

,1670209,1670210,1670211,1670212,1670213
SK_ID_PREV,2300464.00,2.357031e+06,2.659632e+06,2785582.00,2418762.0
SK_ID_CURR,352015.00,3.346350e+05,2.495440e+05,400317.00,261212.0
NAME_CONTRACT_TYPE,1.00,1.000000e+00,1.000000e+00,0.00,0.0
AMT_ANNUITY,14704.29,6.622020e+03,1.152085e+04,18821.52,16431.3
AMT_APPLICATION,267295.50,8.775000e+04,1.052370e+05,180000.00,360000.0
AMT_CREDIT,311400.00,6.429150e+04,1.025235e+05,191880.00,360000.0
AMT_DOWN_PAYMENT,0.00,2.925000e+04,1.052550e+04,NaN,NaN
AMT_GOODS_PRICE,267295.50,8.775000e+04,1.052370e+05,180000.00,360000.0
WEEKDAY_APPR_PROCESS_START,6.00,5.000000e+00,1.000000e+00,6.00,3.0
HOUR_APPR_PROCESS_START,12.00,1.500000e+01,1.200000e+01,9.00,10.0


In [41]:
previous_app['PRODUCT_COMBINATION'].unique()

array([14,  7,  6,  8,  3,  2, 11, 10, 16,  1, 15,  0, 12,  4, 13,  5, 17,
        9])

In [53]:
def get_mode(x):
    return str(x.mode().iloc[0]) 